In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from matplotlib import pyplot as plt
import scipy

## Step 1. Data Collection

In [2]:
# Reading dataset into dta
dta = pd.read_csv('original_training_set/train_manipulated.csv')
dta.shape

(665349, 28)

## Step 2. Splitting data into training set and testing set

In [3]:
train, test = train_test_split(dta, test_size = 0.2)
train.to_csv('new_train_data.csv')
test.to_csv('new_test_data.csv')

In [4]:
dta.isnull().values.sum()

0

## Data of both training set and testing set

In [5]:
train_df1=pd.read_csv('new_train_data.csv')
train_df1.head()

,Unnamed: 0,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,...,B,C,D,E,F,G,cost,weekend_indicator,accident_risk,time_factor
0,389021,10052231,6,1,1,12:56,OH,11175,1,1,...,1,2,2,1,2,2,640,0,3,2
1,390739,10052878,5,0,0,10:37,NY,14397,1,0,...,0,3,1,0,0,1,668,0,4,1
2,118161,10109185,5,0,2,10:59,OH,13634,1,1,...,0,3,3,0,1,3,615,0,3,1
3,547961,10110103,1,0,3,13:23,NM,12219,1,0,...,0,1,1,0,0,4,560,0,3,2
4,410793,10060011,6,0,0,11:39,NY,11797,3,0,...,1,2,3,0,0,3,688,0,4,1


In [6]:
test_df1=pd.read_csv('new_test_data.csv')
test_df1.head()

,Unnamed: 0,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,...,B,C,D,E,F,G,cost,weekend_indicator,accident_risk,time_factor
0,213915,10007330,1,0,1,14:07,TN,11820,1,0,...,1,1,1,1,3,4,666,0,3,2
1,657953,10150119,10,1,1,12:03,PA,13018,1,0,...,0,4,3,1,1,4,673,0,3,2
2,591560,10126079,6,0,0,12:24,OH,14567,1,1,...,1,2,3,1,1,2,595,0,3,2
3,228110,10137889,8,1,3,12:55,WA,10770,2,0,...,1,1,1,0,2,2,659,0,2,2
4,159698,10122864,2,0,2,10:10,PA,10278,1,0,...,1,3,3,1,2,3,656,0,3,1


## Pre-processing data

### Removing the Unnamed column from the new training set

In [7]:
train_df1.shape

(532279, 29)

In [8]:
train_df1.drop(train_df1.columns[0],axis=1, inplace=True)

In [9]:
train_df1.shape

(532279, 28)

In [10]:
train_df1.head()

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,...,B,C,D,E,F,G,cost,weekend_indicator,accident_risk,time_factor
0,10052231,6,1,1,12:56,OH,11175,1,1,1,...,1,2,2,1,2,2,640,0,3,2
1,10052878,5,0,0,10:37,NY,14397,1,0,6,...,0,3,1,0,0,1,668,0,4,1
2,10109185,5,0,2,10:59,OH,13634,1,1,6,...,0,3,3,0,1,3,615,0,3,1
3,10110103,1,0,3,13:23,NM,12219,1,0,0,...,0,1,1,0,0,4,560,0,3,2
4,10060011,6,0,0,11:39,NY,11797,3,0,9,...,1,2,3,0,0,3,688,0,4,1


In [11]:
test_df1.shape

(133070, 29)

In [12]:
test_df1.drop(test_df1.columns[0],axis=1, inplace=True)

In [13]:
test_df1.head()

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,...,B,C,D,E,F,G,cost,weekend_indicator,accident_risk,time_factor
0,10007330,1,0,1,14:07,TN,11820,1,0,2,...,1,1,1,1,3,4,666,0,3,2
1,10150119,10,1,1,12:03,PA,13018,1,0,2,...,0,4,3,1,1,4,673,0,3,2
2,10126079,6,0,0,12:24,OH,14567,1,1,9,...,1,2,3,1,1,2,595,0,3,2
3,10137889,8,1,3,12:55,WA,10770,2,0,4,...,1,1,1,0,2,2,659,0,2,2
4,10122864,2,0,2,10:10,PA,10278,1,0,6,...,1,3,3,1,2,3,656,0,3,1


### Creating response vector and feature set

In [14]:
test_df1.isnull().values.sum()

0

In [15]:
train_df1.columns

Index(['customer_ID', 'shopping_pt', 'record_type', 'day', 'time', 'state',
       'location', 'group_size', 'homeowner', 'car_age', 'car_value',
       'risk_factor', 'age_oldest', 'age_youngest', 'married_couple',
       'C_previous', 'duration_previous', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
       'cost', 'weekend_indicator', 'accident_risk', 'time_factor'],
      dtype='object')

In [16]:
# creating intermediate dataframe which includes feature set of customer_ID and shopping_pt
train_features_df1=pd.DataFrame(np.array(train_df1[['customer_ID','shopping_pt']]) , columns=['customer_ID','shopping_pt'] )
train_features_df1.head()

,customer_ID,shopping_pt
0,10052231,6
1,10052878,5
2,10109185,5
3,10110103,1
4,10060011,6


In [17]:
# creating intermediate dataframe which includes feature set of other required columns

train_features_df2=train_df1.ix[:,7:]
# casting object type to categorical type to convert it later into numeric type
train_features_df2['car_value']=train_features_df2['car_value'].astype('category')

In [18]:
# converting categorical values to numeric values which will be used for prediction
train_features_df2['car_value']=train_features_df2['car_value'].cat.codes

In [19]:
train_features_df2['car_value'].head()

0    4
1    3
2    4
3    5
4    5
Name: car_value, dtype: int8

In [20]:
train_features_df2.head()

,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,C_previous,duration_previous,...,B,C,D,E,F,G,cost,weekend_indicator,accident_risk,time_factor
0,1,1,1,4,4.000000,55,55,0,2,5,...,1,2,2,1,2,2,640,0,3,2
1,1,0,6,3,2.000000,75,75,0,3,2,...,0,3,1,0,0,1,668,0,4,1
2,1,1,6,4,3.099142,26,26,0,3,2,...,0,3,3,0,1,3,615,0,3,1
3,1,0,0,5,4.000000,32,32,0,3,1,...,0,1,1,0,0,4,560,0,3,2
4,3,0,9,5,2.000000,69,30,0,3,15,...,1,2,3,0,0,3,688,0,4,1


In [21]:
# creating response vector to train our model
response_vector=pd.DataFrame(np.array(train_df1['record_type']),columns=['record_type'])
#response_vector.loc[response_vector['record_type']==1]
response_vector.head(30)

,record_type
0,1
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,1


In [22]:
# creating final feature set dataframe
train_features_set=pd.concat([train_features_df1,train_features_df2],axis =1)
train_features_set.shape

(532279, 23)

In [23]:
train_features_set.head(10)

,customer_ID,shopping_pt,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,...,B,C,D,E,F,G,cost,weekend_indicator,accident_risk,time_factor
0,10052231,6,1,1,1,4,4.000000,55,55,0,...,1,2,2,1,2,2,640,0,3,2
1,10052878,5,1,0,6,3,2.000000,75,75,0,...,0,3,1,0,0,1,668,0,4,1
2,10109185,5,1,1,6,4,3.099142,26,26,0,...,0,3,3,0,1,3,615,0,3,1
3,10110103,1,1,0,0,5,4.000000,32,32,0,...,0,1,1,0,0,4,560,0,3,2
4,10060011,6,3,0,9,5,2.000000,69,30,0,...,1,2,3,0,0,3,688,0,4,1
5,10136319,4,2,0,1,4,3.088438,49,46,0,...,1,2,2,1,2,2,673,0,3,3
6,10056494,4,2,1,9,7,3.153409,30,27,1,...,1,3,2,1,3,2,604,0,2,3
7,10109486,1,2,1,2,5,3.020041,43,43,1,...,0,1,1,0,3,1,633,0,2,2
8,10007129,6,1,1,6,4,4.000000,52,52,0,...,0,2,2,0,2,1,632,0,3,1
9,10101540,7,1,1,10,6,3.000000,44,44,0,...,0,1,2,0,2,3,576,0,3,2


In [24]:
#Creating intermediate test features
test_features_df1=pd.DataFrame(np.array(test_df1[['customer_ID','shopping_pt']]) , columns=['customer_ID','shopping_pt'] )
test_features_df1.head()

,customer_ID,shopping_pt
0,10007330,1
1,10150119,10
2,10126079,6
3,10137889,8
4,10122864,2


In [25]:
test_df1.columns

Index(['customer_ID', 'shopping_pt', 'record_type', 'day', 'time', 'state',
       'location', 'group_size', 'homeowner', 'car_age', 'car_value',
       'risk_factor', 'age_oldest', 'age_youngest', 'married_couple',
       'C_previous', 'duration_previous', 'A', 'B', 'C', 'D', 'E', 'F', 'G',
       'cost', 'weekend_indicator', 'accident_risk', 'time_factor'],
      dtype='object')

In [26]:
# creating intermediate dataframe which includes feature set of other required columns
test_features_df2=test_df1.ix[:,7:]
# casting object type to categorical type to convert it later into numeric type
test_features_df2['car_value']=test_features_df2['car_value'].astype('category')

In [27]:
#converting car value from categorical to numeric
test_features_df2['car_value']=test_features_df2['car_value'].cat.codes

In [28]:
# doing the same procedure as above for the test set
test_features_set=pd.concat([test_features_df1,test_features_df2],axis=1)
test_features_set.shape

(133070, 23)

In [29]:
test_features_set.head(15)

,customer_ID,shopping_pt,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,...,B,C,D,E,F,G,cost,weekend_indicator,accident_risk,time_factor
0,10007330,1,1,0,2,5,3.101910,36,36,0,...,1,1,1,1,3,4,666,0,3,2
1,10150119,10,1,0,2,4,3.000000,55,55,0,...,0,4,3,1,1,4,673,0,3,2
2,10126079,6,1,1,9,4,4.000000,65,65,0,...,1,2,3,1,1,2,595,0,3,2
3,10137889,8,2,0,4,4,3.020041,40,33,1,...,1,1,1,0,2,2,659,0,2,2
4,10122864,2,1,0,6,5,3.070130,26,26,0,...,1,3,3,1,2,3,656,0,3,1
5,10016416,3,1,0,1,6,2.000000,44,44,0,...,0,4,3,1,0,2,674,0,3,1
6,10130386,7,2,1,2,4,1.000000,75,70,1,...,1,3,3,1,2,4,623,0,3,1
7,10039082,7,1,1,4,5,1.000000,34,34,0,...,1,3,3,1,2,1,598,0,3,3
8,10044504,4,1,1,17,3,3.000000,22,22,0,...,0,1,2,0,2,3,644,0,4,1
9,10042097,5,1,0,1,4,3.088438,22,22,0,...,0,2,2,0,3,4,670,0,4,2


In [32]:
b=a.fit(train_features_set,response_vector.values.ravel())

TypeError: Cannot clone object '        customer_ID  shopping_pt  record_type  day   time state  location  \
0          10052231            6            1    1  12:56    OH     11175   
1          10052878            5            0    0  10:37    NY     14397   
2          10109185            5            0    2  10:59    OH     13634   
3          10110103            1            0    3  13:23    NM     12219   
4          10060011            6            0    0  11:39    NY     11797   
5          10136319            4            0    4  17:14    OK     15252   
6          10056494            4            0    2  16:56    CO     13136   
7          10109486            1            0    2  15:13    WA     15692   
8          10007129            6            0    3  11:03    IN     11707   
9          10101540            7            1    0  12:50    NM     12663   
10         10019992            5            0    0  11:43    OH     10963   
11         10140243            3            

## Modelling the data to predict the response using Logistic Regression

### Basic Steps to create a model is,
1. Data collection.
2. Data preprocessing:
    1) Data Cleansing.
    2) Data transformation (if required).
    3) Divide data into training and testing sets.
3. Build a model on training data.
4. Evaluate the model on the test data

### Step 1 and  Step 2 are done as shown above

In [33]:
Logistic_Regression_classfifer=LogisticRegression(class_weight='balanced')

In [35]:
x=Logistic_Regression_classfifer.fit(train_features_set,response_vector.values.ravel())

In [38]:
a=SelectFromModel(x, prefit=True)

In [40]:
b=a.transform(train_features_set)
b.shape

(532279, 5)

### Step 3
### b) Fit the model

In [ ]:
classfier_score=Logistic_Regression_classfifer.score(train_features_set,response_vector.values.ravel())
classfier_score

In [ ]:
#test_features_set.sort_values('customer_ID')

In [ ]:
predict_purchase=Logistic_Regression_classfifer.predict(test_features_set)
ctr=Logistic_Regression_classfifer.predict(test_features_set).sum()
ctr

In [ ]:
customer_information = pd.DataFrame(columns=['customer_ID', 'record_type'])
policy_options=test_df1.ix[:,17:24]
customer_information['customer_ID'] = test_df1.customer_ID
customer_information['record_type'] = predict_purchase.astype(int)
predicted_output=pd.concat([customer_information,policy_options],axis=1)
predicted_output.to_csv('logisticregression.csv',index=False)


In [ ]:
test_customer_information=pd.DataFrame(columns=['customer_ID', 'record_type'])
test_customer_information['customer_ID'] = test_df1.customer_ID
test_customer_information['record_type']=test_df1.record_type
expected_output=pd.concat([test_customer_information,test_df1.ix[:,17:24]], axis=1)
expected_output.to_csv('expectedOutput1.csv', index=False)

In [ ]:
lr=pd.read_csv('logisticregression.csv')
lr.head()

In [ ]:
exp=pd.read_csv('expectedOutput1.csv')
exp.loc[exp['record_type']==1].shape

In [ ]:
lr.loc[exp['record_type']==0].shape

In [ ]:
expected=test_df1.record_type


In [ ]:
print(scipy.stats.pearsonr(dta.risk_factor,dta.record_type))

In [ ]:
print(scipy.stats.pearsonr(dta.married_couple,dta.record_type))

In [ ]:
print(scipy.stats.pearsonr(dta.homeowner,dta.record_type))

In [ ]:
print(scipy.stats.pearsonr(dta.homeowner,dta.risk_factor))

In [ ]:
print(recall_score(expected,lr.record_type))

In [ ]:

print (classification_report(expected, lr.record_type))


In [ ]:
print(accuracy_score(expected,lr.record_type))

In [ ]:
print (roc_curve(expected, lr.record_type,pos_label=1))